<a href="https://colab.research.google.com/github/angel870326/Monthly-Revenue/blob/main/002_data_missing_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> 2023.02.28 Ssu-Yun Wang<br/>
[Github @angel870326](https://github.com/angel870326)

# **Update Missing Values from TWSE MOPS**

### Contents

1.  Missing Values
2.  Reformat
3.  Web Crawler Function
    *   每月營收彙總報表 (Not recommended)
    *   個別公司每月營收
4.  Update Missing Values from TWSE MOPS
    *   *For update**
    *   Functions
    *   金融業
    *   非金融業
5.  Output Data


#### Notes

*   若是第一次抓取月營收資料，不要執行 *4.0 For update*
*   再次確認是否有沒抓到的資料，不需要執行 *1. Missing Values* 和 *2. Reformat*，執行 *4.0 For update* 即可






In [1]:
# sConnect to the Google Drive
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
import os
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt

## **1. Missing Values**
From *001_data_partitioning_v1.ipynb*

In [ ]:
# Data path
original_data_path = '/content/gdrive/Shareddrives/Me/論文/資料集/001_v1'

In [ ]:
fin_nan_count1 = pd.read_excel(os.path.join(original_data_path,'上市櫃公司月營收_金融業_中間缺失整理.xlsx'))
nonfin_nan_count1 = pd.read_excel(os.path.join(original_data_path,'上市櫃公司月營收_非金融業_中間缺失整理.xlsx'))
print("金融業", fin_nan_count1.shape)
print("非金融業", nonfin_nan_count1.shape)
fin_nan_count1

金融業 (15, 3)
非金融業 (176, 3)


,公司,Start date,Num of contig NaN
0,5876 上海商銀,1998-06,7
1,5876 上海商銀,1999-06,7
2,5876 上海商銀,2000-06,7
3,5876 上海商銀,2001-06,7
4,5876 上海商銀,2002-06,7
5,5876 上海商銀,2003-06,7
6,5876 上海商銀,2004-06,8
7,5876 上海商銀,2005-06,9
8,5876 上海商銀,2006-06,10
9,5876 上海商銀,2007-06,10


## **2. Reformat**
Create consecutive months from `start date` and `month count`.

fin_nan_month & nonfin_nan_month

In [ ]:
from datetime import datetime

In [ ]:
def createConsecutiveMonth(originalD: pd.DataFrame):
  nan_month = pd.DataFrame(columns = ['公司', '年月', '當月營收（千元）'])
  for index, row in originalD.iterrows():
    dt_list = pd.period_range(start=row['Start date'], periods=row['Num of contig NaN'], freq='M')
    df = pd.DataFrame()
    df['年月'] = dt_list
    df['公司'] = row['公司']
    nan_month = pd.concat([nan_month, df], ignore_index=True, sort=False)
  return nan_month

In [ ]:
fin_nan_month = createConsecutiveMonth(fin_nan_count1)
nonfin_nan_month = createConsecutiveMonth(nonfin_nan_count1)

# Check if there is any error
if len(fin_nan_month) == fin_nan_count1['Num of contig NaN'].sum():
  print('金融業', fin_nan_month.shape)
else:
  print('金融業 (error)')
if len(nonfin_nan_month) == nonfin_nan_count1['Num of contig NaN'].sum():
  print('非金融業', nonfin_nan_month.shape)
else:
  print('非金融業 (error)')

fin_nan_month

金融業 (143, 3)
非金融業 (2643, 3)


,公司,年月,當月營收（千元）
0,5876 上海商銀,1998-06,NaN
1,5876 上海商銀,1998-07,NaN
2,5876 上海商銀,1998-08,NaN
3,5876 上海商銀,1998-09,NaN
4,5876 上海商銀,1998-10,NaN
...,...,...,...
138,6024 群益期,2006-10,NaN
139,6024 群益期,2006-11,NaN
140,6024 群益期,2006-12,NaN
141,6024 群益期,2007-01,NaN


## **3. Web Crawler Function**

### **3.1 每月營收彙總報表 (Not recommended)**

公開資訊觀測站：彙總報表/營運概況/每月營收

2001年6月開始才有資料

In [ ]:
import requests
from io import StringIO

In [ ]:
""" 上市公司（sii）每月營收彙總報表 """

def monthly_report(year, month):   
  # 西元轉民國
  if year > 1990:
      year -= 1911
  
  # URL
  if year >= 102:  
    url = 'https://mops.twse.com.tw/nas/t21/sii/t21sc03_'+str(year)+'_'+str(month)+'_0.html'
  else:
    url = 'https://mops.twse.com.tw/nas/t21/sii/t21sc03_'+str(year)+'_'+str(month)+'.html'
  
  # 偽瀏覽器
  headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    
  # 下載該年月的網站
  r = requests.get(url, headers=headers)
  r.encoding = 'big5'
  dfs = pd.read_html(StringIO(r.text), encoding='big-5')

  # 剃除行數錯誤的表格,並將表格合併
  if year >= 102:
    df = pd.concat([df for df in dfs if df.shape[1] == 11]) 
  else:
    df = pd.concat([df for df in dfs if df.shape[1] <= 11 and df.shape[1] > 5])

  # 設定表格的header 
  df.columns = df.columns.get_level_values(1)

  # 剃除多餘欄位, 重新排序索引值
  df = df[df['公司名稱'] != '合計']
  df = df[df['公司名稱'] != '總計']
  # df['公司'] = df[['公司代號', '公司名稱']].apply(" ".join, axis=1)
  # df = df[['公司', '去年當月營收']]
  df = df.reset_index(drop=True)

  return df

In [ ]:
monthly_report(90, 6)

,公司代號,公司名稱,當月營收,上月營收,去年當月營收,上月比較增減(%),去年同月增減(%),當月累計營收,去年累計營收,前期比較增減(%)
0,1101,台泥,1690526,1769742,1445664,-4.47,16.93,9761724,8642713,12.94
1,1102,亞泥,771532,874662,905595,-11.79,-14.80,4822011,5666028,-14.89
2,1103,嘉泥,179578,167580,223456,7.15,-19.63,1140033,1388661,-17.90
3,1104,環泥,183433,181099,232752,1.28,-21.18,1221780,1526619,-19.96
4,1107,建台,149782,182413,321731,-17.88,-53.44,1346832,2569251,-47.57
...,...,...,...,...,...,...,...,...,...,...
591,9937,全國,747125,695845,607212,7.36,23.04,3919041,3362023,16.56
592,9938,台灣百和,121318,122691,119429,-1.11,1.58,694444,759263,-8.53
593,9939,宏全,137483,130096,121185,5.67,13.44,689615,665701,3.59
594,9940,信義房屋,133864,116664,176174,14.74,-24.01,781371,990542,-21.11


### **3.2 個別公司每月營收**

公開資訊觀測站：營運概況/每月營收<br/>
/採用IFRSs後之月營業收入資訊：https://mops.twse.com.tw/mops/web/t05st10_ifrs<br/>
/採用IFRSs前之開立發票及營業收入資訊(含合併營收)：https://mops.twse.com.tw/mops/web/t05st10<br/>


<br/>

**View POST Data with Chrome Developer Tools**:<br/>
Right-click -> Inspect -> Network -> Payload -> Form Data


In [3]:
import requests
import time

In [4]:
def monthly_revenue(co_id_name, year, month):
  # 公司代號
  co_id = co_id_name.split(' ')[0]

  # 西元轉民國
  if year > 1990:
    year -= 1911
  
  # URL
  if year >= 102:
    url = 'https://mops.twse.com.tw/mops/web/t05st10_ifrs'
    step = 1
  else:
    url = 'https://mops.twse.com.tw/mops/web/t05st10'
    step = 0

  # 月份
  if month < 10:
    month = "0" + str(month)
  else:
    month = str(month)

  r = requests.post(url, {
    'encodeURIComponent': 1,
    'step': step,
    'firstin': 'true',
    'off': 1,
    'isnew': 'false', # 歷史資料
    'co_id': co_id,
    'year': year,
    'month': month,
  })

  r.encoding = 'utf8'
  time.sleep(1) # wait for 1 sec.
  dfs = pd.read_html(r.text, header=None)

  if len(dfs) <= 10:
    # print("No data.")
    df = pd.DataFrame()
  else:
    df = pd.concat([df for df in dfs[10:] if df.shape[1] <= 3 and df.shape[1] >= 2])
    if df.shape[0] > 9: # 採用 IFRSs 前之合併營收
      if "-KY" in co_id_name or "-DR" in co_id_name: # KY 公司
        if df.shape[0] <= 10:
          df.columns = df.iloc[0]
          df = df.iloc[1:]
          df = df.rename(columns = {'合併營業收入淨額':'營業收入淨額'})   
        else:
          df = df.iloc[1:].reset_index(drop=True)
          df.columns = df.iloc[0]
          df = df.iloc[1:][['項目','新台幣']]
          df = df.rename(columns = {'新台幣':'營業收入淨額'})
      else:
        df = pd.concat([df for df in dfs[11:] if df.shape[1] <= 3 and df.shape[1] >= 2])
        df.columns = df.iloc[0]
        df = df.iloc[1:]
        df = df.rename(columns = {'合併營業收入淨額':'營業收入淨額'})

    # df = df[df['項目'] != '增減百分比']
    df = df.reset_index(drop=True)
  
  return df

In [ ]:
monthly_revenue("2330 台積電", 102, 1)

,項目,營業收入淨額
0,本月,47438687.00
1,去年同期,34606486.00
2,增減金額,12832201.00
3,增減百分比,37.08
4,本年累計,47438687.00
5,去年累計,34606486.00
6,增減金額,12832201.00
7,增減百分比,37.08


## **4. Update Missing Values from TWSE MOPS**

採用 3.2 的方法補缺值

### **4.0 For update***
Do not run *1. Missing Values* and *2. Reformat*.


In [3]:
# Updated data path
updated_data_path = '/content/gdrive/Shareddrives/Me/論文/資料集/'

In [4]:
fin_nan_month = pd.read_excel(os.path.join(updated_data_path,'上市櫃公司月營收_金融業_補值.xlsx'))
nonfin_nan_month = pd.read_excel(os.path.join(updated_data_path,'上市櫃公司月營收_非金融業_補值.xlsx'))
print("金融業", fin_nan_month.shape)
print("非金融業", nonfin_nan_month.shape)

金融業 (143, 3)
非金融業 (2643, 3)


In [7]:
# Reformat 年月(str) to period
fin_nan_month['年月'] = pd.to_datetime(fin_nan_month['年月']).dt.to_period('M')
nonfin_nan_month['年月'] = pd.to_datetime(nonfin_nan_month['年月']).dt.to_period('M')

### **4.1 Functions**


In [8]:
def searchMonthlyRevenue(co_id_name: str, year: int, month: int):
  mr = monthly_revenue(co_id_name, year+1, month)
  if mr.empty:
    revenue = np.nan
  else:
    revenue = mr[mr['項目'] == '去年同期'].iloc[0]['營業收入淨額']
  return revenue

In [9]:
def updateMissingData(originalD: pd.DataFrame, sec):
  for index, row in originalD.iterrows():
    if np.isnan(originalD.at[index, '當月營收（千元）']): # if NaN then search
      originalD.at[index, '當月營收（千元）'] = searchMonthlyRevenue(row['公司'], row['年月'].year, row['年月'].month)
      time.sleep(sec) # wait for a few seconds and go on

### **4.2 金融業**

fin_nan_month (143)

In [ ]:
updateMissingData(fin_nan_month, 0.5)
fin_nan_month.isnull().values.sum()

133

### **4.3 非金融業**

nonfin_nan_month (2643)

In [10]:
# Split data into 5 subdata
nonfin_nan_month1 = nonfin_nan_month[:500]
nonfin_nan_month2 = nonfin_nan_month[500:1000]
nonfin_nan_month3 = nonfin_nan_month[1000:1500]
nonfin_nan_month4 = nonfin_nan_month[1500:2000]
nonfin_nan_month5 = nonfin_nan_month[2000:]

In [ ]:
updateMissingData(nonfin_nan_month1, 1)
print("NaN value:", nonfin_nan_month1.isnull().values.sum(), "/", nonfin_nan_month1.shape[0])

NaN value: 465 / 500


In [14]:
updateMissingData(nonfin_nan_month2, 1)
print("NaN value:", nonfin_nan_month2.isnull().values.sum(), "/", nonfin_nan_month2.shape[0])

NaN value: 446 / 500


In [12]:
updateMissingData(nonfin_nan_month3, 1)
print("NaN value:", nonfin_nan_month3.isnull().values.sum(), "/", nonfin_nan_month3.shape[0])

NaN value: 361 / 500


In [ ]:
updateMissingData(nonfin_nan_month4, 1)
print("NaN value:", nonfin_nan_month4.isnull().values.sum(), "/", nonfin_nan_month4.shape[0])

NaN value: 371 / 500


In [ ]:
updateMissingData(nonfin_nan_month5, 1)
print("NaN value:", nonfin_nan_month5.isnull().values.sum(), "/", nonfin_nan_month5.shape[0])

NaN value: 561 / 643


In [13]:
# Concat all the subdata
nonfin_nan_month_update = pd.concat([nonfin_nan_month1, nonfin_nan_month2, nonfin_nan_month3, nonfin_nan_month4, nonfin_nan_month5], ignore_index=True, sort=False)
print("NaN value:", nonfin_nan_month_update.isnull().values.sum(), "/", nonfin_nan_month_update.shape[0])

NaN value: 2204 / 2643


## **5. Output Data**



In [ ]:
# Output data path
output_data_path = '/content/gdrive/Shareddrives/Me/論文/資料集'

In [ ]:
# 金融業：中間有缺失值的公司、年月整理
fin_nan_month.to_excel(os.path.join(output_data_path,'上市櫃公司月營收_金融業_補值.xlsx'), index=False)
print("Data shape:", fin_nan_month.shape)

Data shape: (143, 3)


In [ ]:
# 非金融業：中間有缺失值的公司、年月整理
nonfin_nan_month_update.to_excel(os.path.join(output_data_path,'上市櫃公司月營收_非金融業_補值.xlsx'), index=False)
print("Data shape:", nonfin_nan_month_update.shape)